In [1]:
from __future__ import annotations
from typing import Any, List    

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from torchvision import datasets, transforms 
from nptyping import Number, NDArray

from modules.activation import SquashedTanh
from modules.criterion import MSE
from modules.init import LeCun
from modules.layer import Linear, Conv, Flatten, RBF
from modules.loader import DatasetLoader
from modules.optimizer import Adam
from modules.pooling import AvgPool
from constant import ASCII_BITMAP, C3_MAPPING



In [2]:
composed_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(0, 1)
])

In [3]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=composed_transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=composed_transform)

X_train = np.array([data[0].numpy() for data in train_dataset])
y_train = np.array([data[1] for data in train_dataset])
X_test = np.array([data[0].numpy() for data in test_dataset])
y_test = np.array([data[1] for data in test_dataset])

In [6]:
init_method = LeCun()

class Sequential:
    def __init__(self, blocks: List[Layer]) -> None:
        self.blocks = blocks
        self.is_training = True    

    def forward(
        self, 
        X: NDArray[Any, Number],
        **kwargs
    ) -> NDArray[Any, Number]:
        for block in self.blocks:
            if self.is_training:
                X = block(X, train=self.is_training, y=kwargs['y'])
            else:
                X = block(X, train=self.is_training)
        return X

    def backward(
        self, 
        grad: NDArray[Any, Number]
    ) -> None:
        for block in reversed(self.blocks):
            grad = block.backward(grad)

    def train(self) -> None:
        self.is_training = True

    def eval(self) -> None:
        self.is_training = False
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

In [7]:
model = Sequential([
    # Conv Layers
    Conv((32, 32), 128, 1, 6, 5), #c1
    AvgPool(2), #s2
    SquashedTanh(),
    Conv((14, 14), 128, 6, 16, 5, mapping=C3_MAPPING), #c3
    AvgPool(2), #s4
    SquashedTanh(), 
    Conv((5, 5), 128, 16, 120, 5), #c5
    SquashedTanh(),
    
    # FC Layers
    Flatten(),
    Linear(120, 84, init_method), #f6
    SquashedTanh(),
    RBF(ASCII_BITMAP), #output
])

In [8]:
criterion = MSE()
optimizer = Adam(model, lr=0.001)
batch_size = 128
epochs = 3

In [10]:
loss_list = []
acc_list = []
train_loader = DatasetLoader(X_train, y_train, batch_size=batch_size)
test_loader = DatasetLoader(X_test, y_test, batch_size=batch_size)

for epoch in range(epochs):
    print(f'\nEpoch {epoch + 1}')
    loss = 0
    model.train()

    for X, y in tqdm(train_loader, desc='Training'):
        y_pred = model(X, y=y)
        loss += criterion(y, y_pred)
        grad = criterion.backward()
        model.backward(grad)
        optimizer.step()

    correct = 0
    model.eval()

    for X, y in tqdm(test_loader, desc='Testing'):
        y_pred = model(X)
        correct += np.sum(y_pred == y)

    acc = correct / len(y_test)
    acc_list.append(acc)
    loss_list.append(loss)
    print(f'Accuracy: {acc} | Loss: {loss}')

plt.plot(loss_list)
plt.show()


Epoch 1


Training:   0%|          | 0/468 [00:01<?, ?it/s]


KeyboardInterrupt: 